## Laboratorium 3
**Łukasz Rams**

15.10.2021r

piątek 9:15


**Import modułów**

In [2]:
import psycopg2 as pg
import pandas as pd

**Utworzenie połączenia z bazą danych**

In [3]:
connection = pg.connect(host='pgsql-196447.vipserv.org', 
                        port=5432, 
                        dbname='wbauer_adb', 
                        user='wbauer_adb', 
                        password='adb2020')

**1. Ile kategorii filmów mamy w wypożyczalni?**

In [16]:
df = pd.read_sql('select count(*) from category',con=connection)
print("W wypożyczalni mamy %i kategorii filmów" % int(df["count"]))

W wypożyczalni mamy 16 kategorii filmów


**2. Wyświetl listę kategorii w kolejności alfabetycznej.**

In [30]:
df = pd.read_sql('select name from category order by name',con=connection)
print("Poniżej przedstawiono kategorie w kolejności alfabetycznej\n")
print(df)

Poniżej przedstawiono kategorie w kolejności alfabetycznej

           name
0        Action
1     Animation
2      Children
3      Classics
4        Comedy
5   Documentary
6         Drama
7        Family
8       Foreign
9         Games
10       Horror
11        Music
12          New
13       Sci-Fi
14       Sports
15       Travel


**3. Znajdź najstarszy i najmłodszy film do wypożyczenia.**

In [20]:
df = pd.read_sql('select distinct(release_year) from film', con=connection)
print("Wszystkie filmy są z tego samego roku (%i)" % int(df.release_year))

Wszystkie filmy są z tego samego roku (2006)


**4. Ile wypożyczeń odbyło się między 2005-07-01 a 2005-08-01?**

In [22]:
df = pd.read_sql("select count(*) from rental where rental_date between '2005-07-01' and '2005-08-01'", con=connection)
print("Między 2005-07-01 a 2005-08-01 odbyło się %i wypożyczeń" % int(df["count"]))

Między 2005-07-01 a 2005-08-01 odbyło się 6709 wyporzyczeń


**5. Ile wypożyczeń odbyło się między 2010-01-01 a 2011-02-01?**

In [23]:
df = pd.read_sql("select count(*) from rental where rental_date between '2010-01-01' and '2011-02-01'", con=connection)
print("Między 2010-01-01 a 2011-02-01 odbyło się %i wypożyczeń" % int(df["count"]))

Między 2010-01-01 a 2011-02-01 odbyło się 0 wyporzyczeń


**6. Znajdź największą płatność wypożyczenia.**

In [26]:
df = pd.read_sql("select max(amount) from payment", con=connection)
print("Największa płatność wypożyczenia wynosiła %.2f " % float(df["max"]))

Największa płatność wypożyczenia wynosiła 11.99 


**7. Znajdź wszystkich klientów z Polski, Nigerii lub Bangladeszu.**

In [13]:
# wersja z zagnieżdżaniem select'ów
df = pd.read_sql("""select * 
                    from customer
                    where address_id 
                    in (select address_id 
                        from address
                        where city_id 
                        in(select city_id
                            from city
                            where country_id
                            in (select country_id
                                from country
                                where country
                                in ('Poland', 'Nigeria', 'Bangladesh'))))""", con=connection)


In [29]:
# wersja prostsza
df = pd.read_sql("""select customer.first_name, customer.last_name, country
                    from customer, address, city, country
                    where (customer.address_id = address.address_id)
                    and (address.city_id = city.city_id)
                    and (city.country_id = country.country_id)
                    and (country.country in ('Poland', 'Nigeria', 'Bangladesh'))""", con=connection)
print("Poniższa tabela prezentuje dane klientów z Polski, Nigerii oraz Bangladeszu\n")
print(df)

Poniższa tabela prezentuje dane klientów z Polski, Nigerii oraz Bangladeszu

   first_name  last_name     country
0       Sonia    Gregory     Nigeria
1       Brian      Wyman      Poland
2      Sidney   Burleson      Poland
3      Rodney    Moeller     Nigeria
4     Stephen     Qualls  Bangladesh
5       Velma      Lucas     Nigeria
6     Marilyn       Ross     Nigeria
7       Elsie     Kelley     Nigeria
8      Gladys   Hamilton     Nigeria
9       Frank   Waggoner  Bangladesh
10   Marjorie     Tucker      Poland
11      Carol     Garcia     Nigeria
12       Leah     Curtis      Poland
13      Ruben      Geary      Poland
14       Olga    Jimenez     Nigeria
15     Bertha   Ferguson     Nigeria
16     Tracey    Barrett     Nigeria
17         Jo     Fowler     Nigeria
18    Johnnie   Chisholm      Poland
19    Wallace      Slone     Nigeria
20   Michelle      Clark  Bangladesh
21    Russell    Brinson      Poland
22     Jimmie  Eggleston      Poland
23  Constance       Reid     Nigeri

**8. Gdzie mieszkają członkowie personelu?**

In [31]:
df = pd.read_sql("""select staff.first_name, staff.last_name, address.postal_code, city.city, country.country
                    from staff, address, city, country
                    where (staff.address_id = address.address_id)
                    and (address.city_id = city.city_id)
                    and (city.country_id = country.country_id)""", con=connection)
print("Poniżej przedstawiono gdzie mieszkają członkowie personelu\n")
print(df)

Poniżej przedstawiono gdzie mieszkają członkowie personelu

  first_name last_name postal_code        city    country
0       Mike   Hillyer              Lethbridge     Canada
1        Jon  Stephens               Woodridge  Australia


**9. Ilu pracowników mieszka w Argentynie lub Hiszpanii?**

In [32]:
# Z poprzedniego zadania widać, iż nikt nie mieszka w Argentynie i Hiszpani
df = pd.read_sql("""select count(*)
                    from staff, address, city, country
                    where (staff.address_id = address.address_id)
                    and (address.city_id = city.city_id)
                    and (city.country_id = country.country_id)
                    and (country in ('Argentina', 'Spain'))""", con=connection)
print("W argentynie mieszka %i pracowników" % int(df["count"]))

W argentynie mieszka 0 pracowników


**10. Jakie kategorie filmów zostały wypożyczone przez klientów?**

In [41]:
df = pd.read_sql("""select distinct(category.name )
                    from category, film_category, inventory, rental
                    where (category.category_id = film_category.category_id)
                    and (film_category.film_id = inventory.film_id)
                    and (inventory.inventory_id = rental.inventory_id)
                    and (rental.rental_date is not Null)
                    and (rental.return_date is Null)""", con=connection)
print("Przez klientów zostały wypożyczone i nie zwrócone następujące kategorie filmów\n")
print(df)

Przez klientów zostały wypożyczone i nie zwrócone następujące kategorie filmów

           name
0        Sci-Fi
1        Comedy
2         Drama
3        Horror
4      Classics
5       Foreign
6         Games
7        Travel
8           New
9        Action
10        Music
11  Documentary
12       Sports
13    Animation
14     Children
15       Family


**11. Znajdź wszystkie kategorie filmów wypożyczonych w Ameryce.**

In [63]:
df = pd.read_sql("""select distinct(category.name )
                    from category, film_category, inventory, rental, customer, address, city, country
                    where (category.category_id = film_category.category_id)
                    and (film_category.film_id = inventory.film_id)
                    and (inventory.inventory_id = rental.inventory_id)
                    and (rental.rental_date is not Null)
                    and (rental.return_date is Null)
                    and (rental.customer_id = customer.customer_id)
                    and (customer.address_id = address.address_id)
                    and (address.city_id = city.city_id)
                    and (city.country_id = country.country_id)
                    and (country.country = 'United States')""", con=connection)
print("Przez klientów w Ameryce zostały wypożyczone i nie zwrócone następujące kategorie filmów\n")
print(df)

Przez klientów w Ameryce zostały wypożyczone i nie zwrócone następujące kategorie filmów

          name
0       Action
1    Animation
2     Children
3     Classics
4       Comedy
5  Documentary
6      Foreign
7          New
8       Sports


**12. Znajdź wszystkie tytuły filmów, w których grał: Olympia Pfeiffer lub Julia Zellweger lub Ellen Presley**

In [74]:
df = pd.read_sql("""select film.title, actor.first_name, actor.last_name
                    from film, film_actor, actor
                    where (film.film_id = film_actor.film_id)
                    and (film_actor.actor_id = actor.actor_id)
                    and ((actor.first_name = 'Olympia' and actor.last_name ='Pfeiffer')
                        or (actor.first_name = 'Julia' and actor.last_name ='Zellweger')
                        or (actor.first_name = 'Ellen' and actor.last_name ='Presley'))
                    order by actor.first_name, film.title""", con=connection)
print("Poniżej przedstawiono filmy w których grali Olympia Pfeiffer lub Julia Zellweger lub Ellen Presley\n")
print(df)

Poniżej przedstawiono filmy w których grali Olympia Pfeiffer lub Julia Zellweger lub Ellen Presley

                title first_name last_name
0     Bilko Anonymous      Ellen   Presley
1   Caribbean Liberty      Ellen   Presley
2    Casper Dragonfly      Ellen   Presley
3    Empire Malkovich      Ellen   Presley
4       Floats Garden      Ellen   Presley
..                ...        ...       ...
64  Sweet Brotherhood    Olympia  Pfeiffer
65   Titanic Boondock    Olympia  Pfeiffer
66    Tourist Pelican    Olympia  Pfeiffer
67     Traffic Hobbit    Olympia  Pfeiffer
68         Wait Cider    Olympia  Pfeiffer

[69 rows x 3 columns]
